## Problem koji rešavamo

Zadatak se formuliše kao problem binarne klasifikacije. Cilj analize je da se na osnovu demografskih i kliničkih karakteristika pacijenata predvidi verovatnoću hospitalizacije COVID-19 pacijenata.

Pravovremena procena potrebe za hospitalizacijom može pomoći zdravstvenim ustanovama u planiranju kapaciteta i efikasnijem upravljanju resursima.

Ciljna promenljiva je **HOSPITALIZED**, gde vrednost 1 označava hospitalizovanog pacijenta, dok vrednost 0 označava pacijenta koji nije hospitalizovan.

## SparkML modelovanje – predikcija hospitalizacije

Cilj ovog dela analize je izgradnja modela mašinskog učenja koji može da predvidi da li će COVID-19 pacijent biti hospitalizovan na osnovu demografskih i kliničkih karakteristika.  
Model se implementira korišćenjem SparkML biblioteke.

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (
    SparkSession.builder
    .appName("COVID-SparkML-Hospitalization")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

print("Spark version:", spark.version)
print("Master:", spark.sparkContext.master)

Spark version: 4.1.1
Master: local[*]


### Učitavanje podataka u Spark

U ovom koraku učitavamo prethodno pripremljen dataset u Spark DataFrame koji će se koristiti za dalju ML analizu.

In [15]:
spark_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv('../data/Covid Data.csv')
)

spark_df.printSchema()
spark_df.show(5)

root
 |-- USMER: integer (nullable = true)
 |-- MEDICAL_UNIT: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- PATIENT_TYPE: integer (nullable = true)
 |-- DATE_DIED: string (nullable = true)
 |-- INTUBED: integer (nullable = true)
 |-- PNEUMONIA: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PREGNANT: integer (nullable = true)
 |-- DIABETES: integer (nullable = true)
 |-- COPD: integer (nullable = true)
 |-- ASTHMA: integer (nullable = true)
 |-- INMSUPR: integer (nullable = true)
 |-- HIPERTENSION: integer (nullable = true)
 |-- OTHER_DISEASE: integer (nullable = true)
 |-- CARDIOVASCULAR: integer (nullable = true)
 |-- OBESITY: integer (nullable = true)
 |-- RENAL_CHRONIC: integer (nullable = true)
 |-- TOBACCO: integer (nullable = true)
 |-- CLASIFFICATION_FINAL: integer (nullable = true)
 |-- ICU: integer (nullable = true)

+-----+------------+---+------------+----------+-------+---------+---+--------+--------+----+------+-------+------------

### Kreiranje ciljne promenljive

Originalni dataset ne sadrži eksplicitnu binarnu kolonu hospitalizacije. Zbog toga se ciljna promenljiva **HOSPITALIZED** konstruiše na osnovu kolone **PATIENT_TYPE**.

Vrednost 2 u koloni PATIENT_TYPE označava hospitalizovane pacijente, dok vrednost 1 označava ambulantne pacijente. Ova transformacija omogućava primenu binarne klasifikacije u SparkML okruženju.

In [16]:
from pyspark.sql import functions as F

# Kreiramo labelu HOSPITALIZED iz PATIENT_TYPE
spark_df = spark_df.withColumn(
    "HOSPITALIZED",
    F.when(F.col("PATIENT_TYPE") == 2, 1).otherwise(0)
)

spark_df.select("PATIENT_TYPE", "HOSPITALIZED").show(5)

+------------+------------+
|PATIENT_TYPE|HOSPITALIZED|
+------------+------------+
|           1|           0|
|           1|           0|
|           2|           1|
|           1|           0|
|           1|           0|
+------------+------------+
only showing top 5 rows


### Izbor karakteristika i priprema podataka

Za modelovanje se koriste demografske i kliničke karakteristike pacijenata.

U ovoj fazi vrši se selekcija atributa i uklanjanje nedostajućih vrednosti kako bi se obezbedio kvalitet ulaznih podataka za model mašinskog učenja.

In [18]:
features = [
    "AGE",
    "SEX",
    "PNEUMONIA",
    "DIABETES",
    "HIPERTENSION",
    "OBESITY",
    "RENAL_CHRONIC",
    "COPD"
]

label_col = "HOSPITALIZED"

ml_df = (
    spark_df
    .select(features + [label_col])
    .dropna()
)

ml_df.groupBy(label_col).count().show()

+------------+------+
|HOSPITALIZED| count|
+------------+------+
|           1|200031|
|           0|848544|
+------------+------+



### Podela na trening i test skup

Skup podataka deli se na trening (80%) i test (20%) deo. Model se trenira na trening skupu, dok se test skup koristi za objektivnu procenu performansi na neviđenim podacima.

Ovakav pristup smanjuje rizik od preprilagođavanja (overfitting) modela.

In [19]:
train_df, test_df = ml_df.randomSplit([0.8, 0.2], seed=42)

print("Train size:", train_df.count())
print("Test size:", test_df.count())

Train size: 838937
Test size: 209638


### Kreiranje ML pipeline-a

U cilju standardizacije procesa modelovanja koristi se SparkML pipeline. Numeričke karakteristike objedinjene su pomoću **VectorAssembler** transformacije u jedinstveni vektor obeležja.

Nad tako pripremljenim podacima trenira se model logističke regresije, koji je pogodan za probleme binarne klasifikacije.

In [20]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=features,
    outputCol="features"
)

In [21]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="features",
    labelCol=label_col,
    maxIter=50
)

In [22]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assembler, lr])

model = pipeline.fit(train_df)

### Predikcije modela

In [23]:
pred = model.transform(test_df)

pred.select(
    label_col,
    "prediction",
    "probability"
).show(10, truncate=False)

+------------+----------+----------------------------------------+
|HOSPITALIZED|prediction|probability                             |
+------------+----------+----------------------------------------+
|0           |0.0       |[0.981610889204295,0.018389110795705044]|
|1           |0.0       |[0.981610889204295,0.018389110795705044]|
|1           |0.0       |[0.981610889204295,0.018389110795705044]|
|1           |0.0       |[0.981610889204295,0.018389110795705044]|
|1           |0.0       |[0.981610889204295,0.018389110795705044]|
|1           |0.0       |[0.981610889204295,0.018389110795705044]|
|1           |0.0       |[0.981610889204295,0.018389110795705044]|
|0           |0.0       |[0.981440909882848,0.018559090117151955]|
|0           |0.0       |[0.981440909882848,0.018559090117151955]|
|0           |0.0       |[0.981440909882848,0.018559090117151955]|
+------------+----------+----------------------------------------+
only showing top 10 rows


### Evaluacija modela

Performanse modela procenjuju se korišćenjem više metrika. AUC (Area Under ROC Curve) meri sposobnost modela da razlikuje klase, dok accuracy predstavlja procenat tačno klasifikovanih primera.

Dodatno, konfuziona matrica pruža uvid u tipove grešaka koje model pravi.

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator_auc = BinaryClassificationEvaluator(
    labelCol=label_col,
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

auc = evaluator_auc.evaluate(pred)
print(f"AUC: {auc:.4f}")

AUC: 0.7374


In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator(
    labelCol=label_col,
    metricName="accuracy"
)

accuracy = acc_eval.evaluate(pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8188


In [26]:
pred.groupBy(label_col, "prediction") \
    .count() \
    .orderBy(label_col, "prediction") \
    .show()

+------------+----------+------+
|HOSPITALIZED|prediction| count|
+------------+----------+------+
|           0|       0.0|166304|
|           0|       1.0|  3100|
|           1|       0.0| 34879|
|           1|       1.0|  5355|
+------------+----------+------+

